<div style="text-align: center;">
        <img src="images/molssi_logo_header.png" alt="MolSSI log" style="height: 250px;">
</div>

# Docking with AutoDock Vina

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>
<strong>Questions</strong>

* How can I dock a ligand using AutoDock Vina?
  
* How can I visualize docking structures and ligand interactions?

<strong>Learning Objectives</strong>

* Learn how to define a ligand box for docking.
  
* Execute ligand docking using AutoDock Vina.

* Analyze and interpret docking results using ProLIF.
</div>

In this notebook, we are going to dock the ligands we generated previously notebook with [PDB entry 2zq2](https://www.rcsb.org/structure/2zq2), a trypsin structure from the cow *Bos taurus* that we retrieved and processed in the previous notebook.

Molecular docking simulates the interaction between a small molecule (ligand) and a protein, predicting how these molecules fit together at the molecular level.
It is commonly used in fields such as drug development to design molecules to bind to enzymes or proteins.

During a docking process, AutoDock Vina evaluates numerous potential orientations and conformations of the ligand within the protein’s binding pocket. 
Each pose is assigned a score based on how well it fits, using various scoring functions. 
These functions can be physics-based, empirical, or even be based on machine learning algorithms to predict binding affinity. 
For a recent review of docking scoring functions, you can see [this publication](https://link.springer.com/article/10.1007/S12539-019-00327-W).

In this notebook, we'll be performing docking with the AutoDock Vina Python API using the "vina" scoring function.


### Libraries for the IQB workshop

| Library         | abbreviation | Purpose |
|:-------------|:---------:|:------------|
| os           | N/A      | operating system functions - handling file paths and directories. |
| MDAnalysis     | mda | molecular dynamics library - used for reading/writing files and selecting atoms |
| vina | vina | AutoDock Vina software for Python and Jupyter notebooks |
| prolif | plf | ProLIF (Protein-Ligand Interaction Fingerprints) - generates interaction fingerprints for complexes made of ligands, protein, DNA or RNA molecules


## Preparing for Docking: Defining a Ligand Box

When we dock our ligands to our protein, we will want to define the binding pocket. 
This is where the software will attempt to bind our ligand.
In some cases, people might use software for finding pockets in proteins to define binding sites.
However, as we saw in the previous notebook, the structure we retrieved from the PDB already had a ligand bound.
To define our binding box, we will take the position of the bound ligand from our original structure.

Luckily, MDAnalysis has tools that we can use to measure our molecule and define a binding box. 
The approach we will take in this notebook is to find the `center_of_geometry` of our ligand to define the center of our binding pocket.
Then we will consider the space around the ligand to be our box.

In [4]:
# find the center of the ligand
import MDAnalysis as mda

original_structure = mda.Universe("protein_structures/2zq2.pdb")
ligand_mda = original_structure.select_atoms("resname 13U")

# Get the center of the ligand as the "pocket center"
pocket_center = ligand_mda.center_of_geometry() ## Fill in code for measuring the center of geometry of the ligand.
print(pocket_center)

[18.3068333  -8.06558332 10.638     ]


After defining the pocket center, we will define our ligand box.
One simple approach to this is to subtract the min and max of the ligand positions in each dimension.
In order to allow for ligand flexibility and potential interactions with nearby residues, we will add an additional five angstroms to each side of our box.

In [5]:
# compute min and max coordinates of the ligand
# take the ligand box to be the difference between the max and min in each direction. Plus 5 Angstroms
ligand_box = ligand_mda.positions.max(axis=0) - ligand_mda.positions.min(axis=0) + 5  ## Fill in the rest of this expression.
ligand_box

array([14.161999, 17.002   , 13.903   ], dtype=float32)

The `pocket_center` and `ligand_box` variables are NumPy arrays.
However, AutoDock Vina expects them to be lists.
We convert them to lists in the cell below.

In [6]:
# Convert np array to a list, just changing dtype
pocket_center = pocket_center.tolist()
ligand_box = ligand_box.tolist() ## convert ligand_box to list

<div class="alert alert-block alert-danger">
<strong>Error in the cell above?</strong>  

If you execute the cell above this one more than once, you will see an error occur. 
This happens because on first execution, `pocket_center` and `ligand_box` are NumPy arrays and they have methods to convert to lists.
After you've executed this once, `pocket_center` and `ligand_box` don't have a `tolist` method because they are already lists.

If you execute the cell twice and see an error, you can continue with the rest of the notebook because the variables have already been converted.

</div>

## Docking Ligands with AutoDock Vina

Now that we have PDBQT files of our protein and ligand and have defined our docking box, we are ready to perform the actual docking.
Before docking, we will make a directory to store our results.

In [7]:
# make a directory to store our results
import os

pdb_id = "2zq2"
ligand = "13U"

## Make a directory called "docking_results"
os.makedirs("docking_results", exist_ok=True)

We will dock using the AutoDock Vina Python API.
First, we import `Vina` from `vina`.
We start docking with the line `v = Vina(sf_name="vina")`. 
This creates a docking calculation, `v`, and sets the scoring function to the `vina` scoring function.

In [8]:
from vina import Vina
v = Vina(sf_name="vina")

<div class="alert alert-block alert-success">
<strong>Scoring Functions in AutoDock Vina</strong>

* Vina (`vina`): `vina` is an empirical scoring function. Binding energy is predicted as the sum of pairwise atomic interactions. It includes terms for hydrogen bonds, hydrophobic interactions, and steric interactions. The parameters for this scoring function were empirically derived from fitting data available in the PDBbind database. You can read more in the [original publication](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3041641/), or in the [Vinardo paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4865195/)
* Vinardo (`vinardo`): Vinaro stands for "Vina RaDii Optimized". It was developed to improve the scoring by adjusting atom radii and reparameterizing the empirical terms based on the PDBBIND 2013 database. You can read more in the [Vinardo paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4865195/).
* AutoDock4 (`ad4`):  uses a physics-based model and is the most computationally intensive of the available scores. The `ad4` score requires the definition of a flexible receptor, so it won't work with the PDBQT we have prepared. If you are interested in flexible docking, see [the tutorial from AutoDock Vina](https://autodock-vina.readthedocs.io/en/latest/docking_flexible.html). <strong>If you try to use the `ad4` scoring function on a receptor that was not prepared to be a flexible receptor, your notebook kernel will crash.</strong>

</div>

Then, we set the files for our ligand and receptor. We will dock just our ideal ligand first. There are two parameters to docking, the `exhaustiveness` and `n_poses`.
The exhaustiveness parameter describes the "exhaustiveness" of the docking - a higher exhaustiveness means that more ligand conformations are tried. Exhaustiveness also corresponds to the amount of computational effort used during a docking experiment. The default exhaustiveness value is 8; increasing this to 32 will give a more consistent docking result. 

In this notebook, we set the exhaustiveness to 5 to improve speed for the workshop. If you were to do a real docking calculation, you should consider increasing this parameter. You may also consider looking into Smina for custom scoring.

In [9]:
v.set_receptor(f"pdbqt/{pdb_id}.pdbqt")   # Set the receptor created earlier
v.set_ligand_from_file(f"pdbqt/{ligand}.pdbqt") ## Fill in the ligand name
v.compute_vina_maps(center=pocket_center, box_size=ligand_box) ## Fill in the ligand center and box size calculated previously.

Computing Vina grid ... done.


In [10]:
# Step that does the actual docking
v.dock(exhaustiveness=5, n_poses=5)

Performing docking (random seed: -1305448317) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


After the `dock` function, we can write the poses that were calculated to a file.
Note that the output format from AutoDock Vina is a PDBQT file.

In [11]:
# Note that it has the same name as the original ligand .pdbqt, can consider writing to new name such as "_out.pdbqt"
v.write_poses(f"docking_results/{ligand}.pdbqt", n_poses=5, overwrite=True)

We can see the energies of the calculated poses by calling `energies` on the docking calculation variable.
According to the Vina documentaiton, the rows correspond to the poses, while columns correspond to different energy types.
The types of energies in the columns are `["total", "inter", "intra", "torsions", "intra best pose"]`.
The number of columns and the types of energies they represent depend on the scoring function you are using.
You can see more information in the [docs for AutoDock Vina](https://autodock-vina.readthedocs.io/en/latest/vina.html#vina.vina.Vina.energies).

In [12]:
## Fill in code to see the calculated energies for the poses.
# Produces a np array with energies correspond to ["total", "inter", "intra", "torsions", "intra best pose"]
v.energies()

array([[-5.159, -9.684, -0.311,  4.524, -0.311],
       [-5.09 , -8.51 , -1.354,  4.463, -0.311],
       [-5.033, -9.28 , -0.477,  4.413, -0.311],
       [-4.693, -8.482, -0.637,  4.115, -0.311],
       [-4.256, -8.116, -0.183,  3.732, -0.311]])

You might wish to save these energies to return to them later. 
The cell below creates a pandas dataframe and saves the energies as a comma-separated-value (CSV) file.

In [13]:
import pandas as pd


# These are the columns for the types of energies according to AutoDock Vina docs.
column_names = ["total", "inter", "intra", "torsions", "intra best pose"]

df = pd.DataFrame(v.energies(), columns=column_names) ## Fill in arguments to the dataframe to create a dataframe with energies.
df.head()

,total,inter,intra,torsions,intra best pose
0,-5.159,-9.684,-0.311,4.524,-0.311
1,-5.090,-8.510,-1.354,4.463,-0.311
2,-5.033,-9.280,-0.477,4.413,-0.311
3,-4.693,-8.482,-0.637,4.115,-0.311
4,-4.256,-8.116,-0.183,3.732,-0.311


In [14]:
# Save the calculated energies from docking to a CSV file
df.to_csv("docking_results/13U_energies.csv", index=False) ## Fill in file name.

## Visualizing Docking Results

After performing the docking simulation and saving the energies, you might wish to visualize the poses. 
When visualizing results from molecular docking, scientists often visually inspect the 3D docked structure as well as a 2D representation called an interaction map.
We can ues a software called ProLIF (Protein-Ligand Interaction Fingerprints) to make and view these maps in the Jupyter notebook.

To generate these visualizations, we have to convert our files (again!) to the correct format.

In the step above, we wrote the poses to the file `docking_results/13U.pdbqt`. 
AutoDock Vina only writes in this file, but in order to visualize your results, we need a more standard format.
We will use meeko again to convert our poses to an SDF.
Note that meeko will only convert pdbqt files if it prepared the input docking files, which is one reason we used it in the previous notebook.

Again, we use a command line script to convert out poses.

In [15]:
! mk_export.py docking_results/13U.pdbqt -o docking_results/13U.sdf

After converting to SDF, we can again visualize our results with ProLIF.
ProLIF requires that molecules be loaded in and has functions to load molecules in several ways.
We will use MDAnalysis for loading our proteins to ProLIF and `sdf_supplier` to load the SDFs we converted in the previous step.

In [16]:
import prolif as plf
import MDAnalysis as mda

pdb_id = "2zq2"   # Use structure with hydrogens to look at interactions just like Vina

protein = mda.Universe(f"protein_structures/protein_h.pdb")

Next, we load our protein and ligand into ProLIF.
The function we use to do this depends on the format of our input data.

In [18]:
protein_plf = plf.Molecule.from_mda(protein)
poses_plf = plf.sdf_supplier("docking_results/13U.sdf") # fill in filename here

To analyze the interactions of the ligand and protein we create a molecular fingerprint object.
By default, ProLIF will calculate nine types of interactions: 'Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'
However, you could set this to different interactions. You can see more information about the types of interactions in the [ProLIF docs](https://prolif.readthedocs.io/en/latest/source/modules/interaction-fingerprint.html#detecting-interactions-between-residues-prolif-interactions-interactions).

In [19]:
fp = plf.Fingerprint(count=True)

Next, we will run ProLif on our poses.
To do this calculation, we pass in our list of poses (`poses_plf`) and our ProLIF protein.

In [20]:
# run on your poses, pass poses and protein as argument
fp.run_from_iterable(poses_plf, protein_plf) ## fill in this function

  0%|          | 0/5 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 9 interactions: ['Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'] at 0x789b604c6010>

After running this analysis, we can visualize the interaction results.
We are using the 2D and 3D visualization maps here, but there are [many other types of analysis](https://prolif.readthedocs.io/en/latest/notebooks/docking.html#analysis) that you can perform.

In [34]:
# Since there are 5 poses you can use others [0-4], index 0
pose_index=0

In [35]:
fp.plot_lignetwork(poses_plf[pose_index])

In [37]:
view = fp.plot_3d(
    poses_plf[pose_index], protein_plf, frame=pose_index, display_all=False
)
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

<div class="alert alert-block alert-warning"> 
<h3>Exercise</h3>

Try docking one of the ligands we modified in the previous notebook. Does it bind better or worse according to the docking score? Are the interactions different for the poses?
</div>

In [38]:
# make a directory to store our results
import os

pdb_id = "2zq2"
ligand_N = "13U_modified_N"

## Make a directory called "docking_results"
os.makedirs("docking_results", exist_ok=True)

In [39]:
from vina import Vina
v = Vina(sf_name="vina")

In [40]:
v2.set_receptor(f"pdbqt/{pdb_id}.pdbqt")   # Set the receptor created earlier
v2.set_ligand_from_file(f"pdbqt/{ligand_N}.pdbqt") ## Fill in the ligand name
v2.compute_vina_maps(center=pocket_center, box_size=ligand_box) ## Fill in the ligand center and box size calculated previously.


mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -5.159          0          0
   2        -5.09      2.828      6.894
   3       -5.033      1.594      2.239
   4       -4.693      2.843      6.696
   5       -4.256      3.573      6.036
Computing Vina grid ... done.


In [41]:
# Step that does the actual docking
v.dock(exhaustiveness=8, n_poses=6)

Performing docking (random seed: 1431737464) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [43]:
# Note that it has the same name as the original ligand .pdbqt, can consider writing to new name such as "_out.pdbqt"
v.write_poses(f"docking_results/{ligand_N}.pdbqt", n_poses=5, overwrite=True)